In [ ]:
!pip install langchain-mcp-adapters langgraph langchain-openai

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from subprocess import check_output

In [ ]:
# enable multi-turn conversations
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

config = {"configurable": {"thread_id": "1"}}

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    openai_api_base = "https://litellm.lib.ou.edu/",
    openai_api_key = check_output(["pass", "application/ul-llm/tdp"]).strip(),
    model = "nova-lite-v1:0",
    #openai_api_base = "http://ellm.nrp-nautilus.io/v1",
    #openai_api_key = check_output(["pass", "application/nrp-llm/ul-tdp"]).strip(),
    #model = "olmo",
    temperature=0.1
)


In [ ]:
from pathlib import Path

server_params = StdioServerParameters(
    command="python",
    args=[str(Path.cwd() / "pg_server_SHAREOK.py")],
)

In [ ]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools, checkpointer=checkpointer)
        #agent_response = await agent.ainvoke({"messages": "What papers are about machine learning? Give me the author name, title, two sentence summary of the abstract, and the url."}, config)
        agent_response = await agent.ainvoke({"messages": "What papers are about machine learning? Give me the title and url."}, config)

In [ ]:
from IPython.display import Markdown

results = agent_response['messages'][-1].content

Markdown(results)

In [ ]:
agent_response = await agent.ainvoke({"messages": "Give me a list of any other papers you found."}, config)
results = agent_response['messages'][-1].content

Markdown(results)

In [ ]:
agent_response = await agent.ainvoke({"messages": "Do any of these papers mention topics on graph theory?"}, config)
results = agent_response['messages'][-1].content

Markdown(results)